In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:

import torch.nn as nn

from model.JEPA import ExploreJEPA
from dataset import create_wall_dataloader

In [3]:
train_loader = create_wall_dataloader("/scratch/DL25SP/train")

In [4]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [5]:
model = ExploreJEPA(image_size=65,
                    encoding_hidden_dim=16,
                    encoding_dim=256,
                    encoding_layers=3)
model.to("cuda")

ExploreJEPA(
  (init_state_encoder): StateEncoder(
    (agent_pwConv): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (agent_blocks): Sequential(
      (0): ConvNextBlock(
        (dwConv): Conv2d(16, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=16, bias=False)
        (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (pwConv4): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gelu): GELU(approximate='none')
        (pwConv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): ConvNextBlock(
        (dwConv): Conv2d(16, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=16, bias=False)
        (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (pwConv4): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gelu): GELU(approximate='none')
        (pwConv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (2): ConvN

In [6]:
criterion = nn.MSELoss()

for data in train_loader:
    states = data.states
    actions = data.actions
    p_s, e_s = model(states, actions)
    c = criterion(p_s, e_s)
    reg = torch.mean(p_s.var(dim=1)) + torch.mean(e_s.var(dim=1))
    energy = c + reg
    energy.backward()
    
    break

/scratch/xy2053/2025SP/2572_DeepLearning/codes/dataset.py:32: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  states = torch.from_numpy(self.states[i]).float().to(self.device)


In [3]:
from train import training_loop

In [4]:
epoch = 10
batch_size = 64

learning_rate = 1e-4
weight_decay = 1e-5

train_dataloader = create_wall_dataloader(data_path="/scratch/DL25SP/train", batch_size=batch_size)

model = ExploreJEPA(image_size=65,
                    encoding_hidden_dim=16,
                    encoding_dim=256,
                    encoding_layers=3)
model.to("cuda")

train_model = training_loop(model, epoch, learning_rate, weight_decay, train_dataloader)

  0%|          | 0/2297 [00:00<?, ?it/s]/scratch/xy2053/2025SP/2572_DeepLearning/codes/dataset.py:32: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  states = torch.from_numpy(self.states[i]).float().to(self.device)
  4%|▍         | 100/2297 [01:53<41:33,  1.14s/it, total energy: =4.54, mse loss:=6.29, reg loss:=2.79]


KeyboardInterrupt: 